In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import BitsAndBytesConfig
import torch
from peft import PeftModel

base_model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

nf4Config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=False,
                                           trust_remote_code=True,
                                           add_eos_token=True)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=nf4Config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

modelFinetuned = PeftModel.from_pretrained(base_model,"/home/spalaniappan9/nlp/project/weights/finetunedModel/checkpoint-6135")

# model = modelFinetuned
model = base_model
tokenizer.pad_token = tokenizer.eos_token

/home/spalaniappan9/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/spalaniappan9/.venv/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:21<00:00,  5.28s/it]


: 

In [ ]:
# ENGLISH

import pandas as pd
import csv

df = pd.read_csv("englishQuestions.csv")
questions = df['How is synthesized speech created?'].tolist()
for question in questions: 
    
    messages = [
    {"role": "system", 
        "content": "Be a helpful assistant and answer the user's questions; be precise, and concise"},
    {"role": "user", 
        "content": f"{question}"}
    ]

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        # device="cuda"
    )

    generation_args = {
        "max_new_tokens": 600,
        "return_full_text": False,
        "temperature": 0.2,
    }

    output = pipe(messages, **generation_args)
    output_text = output[0]['generated_text']
    output_text = output_text.replace("output:", "").strip()
    if "None" in output_text:
        output_text = "None"
    output_file = "englishAnswers.csv"
    if os.path.exists(output_file):
        with open(output_file, mode='a') as f:
            writer = csv.writer(f)
            writer.writerow([output_text])
    else:
        with open(output_file, mode='w') as f:
            writer = csv.writer(f)
            writer.writerow([output_text])

In [ ]:
# TAMIL

import pandas as pd
import csv

df = pd.read_csv("tamilQuestions.csv")
questions = df['தொகுக்கப்பட்ட பேச்சு எவ்வாறு உருவாக்கப்படுகிறது?'].tolist()
for question in questions: 
    
    messages = [
    {"role": "system", 
        "content": "Be a helpful assistant and answer the user's questions in Tamil; be precise, and concise"},
    {"role": "user", 
        "content": f"{question}"}
    ]

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        # device="cuda"
    )

    generation_args = {
        "max_new_tokens": 600,
        "return_full_text": False,
        "temperature": 0.2,
    }

    output = pipe(messages, **generation_args)
    output_text = output[0]['generated_text']
    output_text = output_text.replace("output:", "").strip()
    if "None" in output_text:
        output_text = "None"
    output_file = "tamilAnswers.csv"
    if os.path.exists(output_file):
        with open(output_file, mode='a') as f:
            writer = csv.writer(f)
            writer.writerow([output_text])
    else:
        with open(output_file, mode='w') as f:
            writer = csv.writer(f)
            writer.writerow([output_text])